In [281]:
import numpy as np
import pandas as pd

RI = [
    0, 0, 0.52, 0.89, 1.12, 1.26, 1.36, 1.41, 1.46, 1.49, 1.52, 1.54, 1.56,
    1.58, 1.59, 1.60, 1.61, 1.615, 1.62, 1.63
]


class AHPmodel:
    """层次分析法"""
    def __init__(self, A=None, B=None):
        """初始化模型"""
        self.A = A
        self.B = B

    @staticmethod
    def test(matrix):
        """一致性检验"""
        n = matrix.shape[0]

        # D:特征值 V:特征向量
        D, V = np.linalg.eig(matrix)

        # 寻找最大特征值及其特征向量
        Dmax = -np.inf
        for i in range(n):
            if D[i] > Dmax:
                Dmax = D[i]
                pos = i
        t = np.around(Dmax, decimals=4).real
        w = V[:, pos]

        # 归一化
        w = np.around(np.abs(w) / np.sum(np.abs(w)), 4)

        # 检验
        CI = np.around((t - n) / (n - 1), 4)
        CR = np.around(CI / RI[n - 1], 4)
        if CR < 0.1:
            pass_test = True
        else:
            pass_test = False

        return t, w, CI, CR, pass_test

    def start(self):
        """进行计算"""
        try:
            n = self.A.shape[0]
            m = self.B[0].shape[0]

            # 对A进行一致性检验并保存结果
            A_result = self.test(self.A)
            A_result_set = pd.Series(data=A_result,
                                     index=['tmax', 'w', 'CI', 'CR', 'pass'])

            # 对B进行一致性检验并保存结果
            B_result_set = pd.DataFrame(
                columns=['t_max', 'w', 'CI', 'CR', 'pass'])
            for i in range(n):
                Bi_result = [self.test(self.B[i])]
                Bi_result = pd.DataFrame(
                    data=Bi_result, columns=['t_max', 'w', 'CI', 'CR', 'pass'])
                B_result_set = pd.concat([B_result_set, Bi_result],
                                         ignore_index=True)
            B_result_set.index = [f"因素{i+1}" for i in range(n)]

            # 总的一致性检验
            w = np.array(A_result_set['w']).reshape(5, 1)
            CI = np.array(B_result_set['CI'])
            CR = np.around((np.dot(CI, w) / np.sum(w * RI[m - 1]))[0], 4)
            pass_last_test = False
            if CR < 0.1:
                pass_last_test = True
                Wi = []
                for i in range(n):
                    Wi.append(B_result_set['w'][i])
                Wi = np.array(Wi).T
                result = np.around(np.dot(Wi, w), 4)
                result_set = pd.Series({
                    'w': result,
                    'CR': CR,
                    'pass': pass_last_test
                })
                self.A_result_set = A_result_set
                self.B_result_set = B_result_set
                self.result_set = result_set
                return result_set

        except:
            print("请正确初始化因素比较矩阵A和方案比较矩阵列表B！！！")

    def show(self):
        """展示结果"""
        print("因素层结果显示:")
        print(self.A_result_set)
        print("\n方案层结果显示:")
        print(self.B_result_set)
        print("\n最终结果显示:")
        print(self.result_set)

        w = self.result_set['w']
        m = w.size
        w_max = -np.inf
        pos = 0
        for i in range(m):
            if w[i] > w_max:
                pos = i
        print(f"\n根据层次分析法,应选择方案{pos+1}")

In [282]:
A = np.array([[1, 1 / 2, 4, 3, 3], [2, 1, 7, 5, 5],
              [1 / 4, 1 / 7, 1, 1 / 2, 1 / 3], [1 / 3, 1 / 5, 2, 1, 1],
              [1 / 3, 1 / 5, 3, 1, 1]])
B1 = [[1, 2, 5], [1 / 2, 1, 2], [1 / 5, 1 / 2, 1]]
B2 = [[1, 1 / 3, 1 / 8], [3, 1, 1 / 3], [8, 3, 1]]
B3 = [[1, 1, 3], [1, 1, 3], [1 / 3, 1 / 3, 1]]
B4 = [[1, 3, 4], [1 / 3, 1, 1], [1 / 4, 1, 1]]
B5 = [[1, 1, 1 / 4], [1, 1, 1 / 4], [4, 4, 1]]
B = np.array([B1, B2, B3, B4, B5])
pro = AHPmodel()
pro.A = A
pro.B = B
pro.start()
pro.show()

因素层结果显示:
tmax                                      5.0721
w       [0.2636, 0.4758, 0.0538, 0.0981, 0.1087]
CI                                         0.018
CR                                        0.0161
pass                                        True
dtype: object

方案层结果显示:
      t_max                         w      CI      CR  pass
因素1  3.0055  [0.5954, 0.2764, 0.1283]  0.0028  0.0054  True
因素2  3.0015  [0.0819, 0.2363, 0.6817]  0.0008  0.0015  True
因素3     3.0  [0.4286, 0.4286, 0.1429]     0.0     0.0  True
因素4  3.0092  [0.6337, 0.1919, 0.1744]  0.0046  0.0088  True
因素5     3.0  [0.1667, 0.1667, 0.6667]     0.0     0.0  True

最终结果显示:
w       [[0.2993], [0.2453], [0.4554]]
CR                               0.003
pass                              True
dtype: object

根据层次分析法,应选择方案3
